# Refactoring

지치지 말고 잘 하자..!

## 발생한 문제

- 01 Logging이 중복으로 발생하는 이슈
- 02 데이터 Scaling Encoder가 증강된 데이터셋에 알맞게 처리가 안 됨
- 03 Loss, Optimizer, Scheduler 잘 적용 가능 하도록 Refactoring
- 04 Refactoring을 진행하면서 README 수정도 해야 된다.
- 05 TestDataset Result가 제대로 생성이 안된다.

## 01 Logging이 중복으로 발생하는 이슈

- Logger가 Trainer, Preprocess 생성될 때 마다 실행된다.
- Module에서 전역으로 코드를 불러와서 그런 듯

해결 여부 : [O] 

## 02 데이터 Scaling Encoder가 증강된 데이터셋에 알맞게 처리가 안 됨

- Encoder의 Fit이 Train Dataset에 맞게만 추가 된후 TestDataset이 추가 되어서 발생한다.
- 전처리 과정 순서를 바꿔야 된다.

**지금 전처리 과정**

1. Feature Engineering
2. Split Data
    - train, test => train, valid, test
3. Scaling
    - TrainData에 Fit 맞춘 후 Transform 과정 수행
    - TestData에 Transform 과정 수행
4. Data Augmentation
    1. (사용) 테스트 데이터셋 추가
    2. 사이클 증강 group by ("userID", "testPaper")
    3. (사용) 대분류 증강 group by ("userID", "firstClass")
    
**바뀐 후 전처리 과정**

1. 테스트 데이터셋 추가
2. Feature Engineering  # valid dataset을 포함하여 Fit을 하는 것이 맞을까? => 맞지 않다.
    - Test 데이터 셋의 거의 전부를 포함해서 보기 때문에 괜찮을 것 같다.
3. Scaling
    - Test가 Train 데이터 셋에 포함이 안 됐다면 Split Data먼저 했을 텐데...
    - 거의 포함되니깐 Scaling을 하는게 좋을 것 같다.
4. Split Data
    - train은 테스트 데이터 셋 추가되고, Train 데이터 셋은 거의 모든 데이터셋이 포함된 데이터 셋
    - train, test => train, valid, test
5. Data Augmentation
    - (사용) 대분류 증강 group by ("userID", "firstClass")
    - 사이클 증강 group by ("userID", "testPaper")
    
`중간 중간 UserID가 정렬이 잘 됐는지 체크한다...`

- 순서 바뀌면서 미리 생성된 Feature들 다 오작동 ... ㅠㅜㅠㅜ


**실험 중인 전처리 과정**

- Feature Engineering 과정은 Train, Valid 따로 따로 진행하는 것
    - 의심 : Valid의 Feature에서 정보 유출이 진행되는 것 같다. 

In [1]:
import sys
import os.path as p

sys.path.append("/home/j-gunmo/desktop/00.my-project/17.P-Stage-T1003/4-STAGE/")

from fe.feature import FEBase, FEPipeline

from fe.agg import (
    MakeCorrectCount, 
    MakeCorrectPercent, 
    MakeQuestionCount, 
    MakeTopNCorrectPercent
)

from fe.seq import (
    SplitAssessmentItemID,
    MakeFirstClass,
    MakeSecondClass,
    MakeTimeDiff,
    MakeYMD,
    ConvertTime
)

import easydict
from IPython.display import clear_output

from dkt_dataset import Preprocess
from utils import get_args, get_root_dir

In [2]:
args = get_args()
args.data_dir = "../../input/data/train_dataset/"
args.root_dir = get_root_dir("../refactoring/")

In [3]:
fe_pipeline = FEPipeline(
    args, [
        SplitAssessmentItemID,
        ConvertTime,
        MakeFirstClass,
        MakeSecondClass,
        MakeCorrectCount,
        MakeQuestionCount,
        MakeCorrectPercent
    ]
)

columns = [
    "userID",
    "answerCode",
    "testPaper",
    "timeSec",
    "firstClass",
    "secondClass",
    "correctPer"
]

pre_encoders = {
    "label": ["testPaper", "firstClass", "secondClass"],
    "min_max": ["correctPer"],
    "std": ["timeSec"],
}

In [4]:
preprocess = Preprocess(args, fe_pipeline, columns)

In [5]:
# 테스트 데이터셋 추가
preprocess._data_augmentation_test_dataset()

Use the test datast for data augmentation


Before Length: 2266586


After Length: 2525956


In [6]:
user = preprocess.datas['train'].userID.unique()
len(user), max(user), min(user)  # 0부터 시작

(7442, 7441, 0)

In [7]:
# Feature Engineering -> 기존에 생성된 Feature들 Shape이 안 맞아서 폐기처리 ㅠㅜ
preprocess.feature_engineering()

Feature Engineering Start ... 


load features /home/j-gunmo/features/train_split_assessmentitem_id.pkl to dataframe ... 



Feature Engineering Name: split_assessmentitem_id



testPaper       : 시험지 번호입니다.


dtype: object


...

INDEX 8000: 0.6911196911196911


INDEX 9000: 0.6911196911196911


Feature Engineering End ... 


Original DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag'],
      dtype='object')


Feature Added DataFrame Keywords: Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'testPaper', 'testPaperCnt', 'timeSec', 'firstClass',
       'secondClass', 'correctCnt', 'quesCnt', 'correctPer'],
      dtype='object')


In [8]:
preprocess.datas['train'].head(2)

,userID,answerCode,testPaper,timeSec,firstClass,secondClass,correctPer
0,0,1,060001,1584976631,6,7224,0.630872
1,0,1,060001,1584976634,6,7225,0.630872


In [9]:
# Scaling
preprocess.scaling(pre_encoders)

Preprocessing Labels .. 


Label Columns: ['testPaper', 'firstClass', 'secondClass']



Length of testPaper            : 1538


Before : 0    060001
1    060001
2    060001
3    060001
4    060001
5    060001
6    060003
7    060003
8    060003
9    060003
Name: testPaper, dtype: object


After : 0    975
1    975
2    975
3    975
4    975
5    975
6    977
7    977
8    977
9    977
Name: testPaper, dtype: int64


...

Min Max Columns: ['correctPer']


MAX: [1.] MIN: [0.]


Preprocessing Min Max .. 


Standard Columns: ['timeSec']


MEAN: [1.59505272e+09] VAR: [4.91248814e+13]


In [10]:
preprocess.datas['train'].head(2)

,userID,answerCode,testPaper,timeSec,firstClass,secondClass,correctPer
0,0,1,975,-1.437611,5,618,0.630872
1,0,1,975,-1.437610,5,619,0.630872


In [11]:
# Split Data
preprocess.split_data()

Split based on User


Original Train Dataset: 2525956


Split Train Dataset: 2264710


Split Valid Dataset: 261246


In [12]:
# Data Augmentation
preprocess._data_augmentation_user_testid()

Group By (userID, firstClass)


Group By (userID, firstClass) Length: 17776


Group By (userID, firstClass) Length: 1963


Group By (userID, firstClass) Length: 1987


### 02-01 BaseLine 모델이랑 비교

- 같은 Feature 사용 (02-make_feature랑 비교)
- AUC : 0.7672
- ACC : 0.7218

In [19]:
import torch
from models.lstm.model import LSTM
from trainer import DKTTrainer

class FeatureTestTrainer(DKTTrainer):
    def _process_batch(self, batch):
        batch['mask'] = batch['mask'].type(torch.FloatTensor)
        batch["answerCode"] = batch["answerCode"].type(torch.FloatTensor)

        batch["interaction"] = batch["answerCode"] + 1
        batch["interaction"] = batch["interaction"].roll(shifts=1, dims=1)
        batch["mask"] = batch["mask"].roll(shifts=1, dims=1)
        batch["mask"][:, 0] = 0
        batch["interaction"] = (batch["interaction"] * batch["mask"]).to(torch.int64)
        
        
        for k in args.n_linears: # 수치형
            batch[k] = batch[k].type(torch.FloatTensor)
            
        for k, v in args.n_embeddings.items(): # 범주형
            batch[k] = batch[k].to(torch.int64)
            
        for k in batch.keys():
            batch[k] = batch[k].to(self.args.device)
        
        return batch

def log_notebook_fold5(trainer, train_data, valid_data, test_data):
    auc, acc = trainer.run_cv(train_data, valid_data, test_data, 
                              folds=5, seeds=[0, 1, 2, 3, 4])
    clear_output()
    print(f"auc: {auc}, acc: {acc}")
    
def log_notebook_fold10(trainer, train_data, valid_data, test_data):
    auc, acc = trainer.run_cv(train_data, valid_data, test_data, 
                              folds=10, seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    clear_output()
    print(f"auc: {auc}, acc: {acc}")

In [13]:
args.columns = columns[1:]
train_data = preprocess.get_data("train_grouped")
valid_data = preprocess.get_data("valid_grouped")
test_data = preprocess.get_data("test_grouped")

In [14]:
args.num_workers = 2
args.n_epochs = 10
args.hidden_dim = 512
args.use_dynamic = True

In [17]:
trainer = FeatureTestTrainer(args, LSTM)

In [18]:
log_notebook_fold5(trainer, train_data, valid_data, test_data)
print(f"logging path : {trainer.prefix_save_path}")

auc: 0.76686559031009, acc: 0.6900709219858158
logging path : ../refactoring/LOG_[06.10_10:07]


In [23]:
args.num_workers = 2
args.n_epochs = 10
args.hidden_dim = 1024
args.use_dynamic = True

trainer = FeatureTestTrainer(args, LSTM)

In [24]:
log_notebook_fold10(trainer, train_data, valid_data, test_data)
print(f"logging path : {trainer.prefix_save_path}")

auc: 0.7647756599363843, acc: 0.6888044579533943
logging path : ../refactoring/LOG_[06.10_10:53]


auc 가 올랐다!!!!

## 03 Loss, Optimizer, Scheduler 잘 적용 가능 하도록 Refactoring

- HyperParameter Tuning을 위해서 필요하다.
- Loss, Optimizer, Scheduler에 대해서 상세한 이해 후에 사용을 해야 된다.
    - Default 값을 설정을 잘해야 된다.
- loss.forward
- optimizer.step
- scheduler.step

## 04 Refactoring을 진행하면서 README 수정도 해야 된다.

- Feature Engineering README부터 작성

## 05 TestDataset Result가 제대로 생성이 안된다.

- 이건 아직 이유를 잘 모르겠음... Inference에서 오류가 나나
- use_dynamic 때문이였음
- 해결 여부 [O]